Connect to MongoDB

In [10]:
from pymongo import MongoClient
import pprint

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")  # Adjust if needed
db = client["TripAdvisor"]  # Select the database
collection = db["restaurants"]  # Select the collection

# Pretty Print
pp = pprint.PrettyPrinter(indent=2)



Copy File to Container 

In [11]:
import subprocess
# Define variables
local_path = r"E:\ModifiedDownloads\tripadvisor_european_restaurants.json"
container_path = "/tmp/tripadvisor_european_restaurants.json"
container_name = "MONGO_CONTAINER"  # Replace with actual container name

# Copy file from host to Docker container
subprocess.run(["docker", "cp", local_path, f"{container_name}:{container_path}"], check=True)


CompletedProcess(args=['docker', 'cp', 'E:\\ModifiedDownloads\\tripadvisor_european_restaurants.json', 'MONGO_CONTAINER:/tmp/tripadvisor_european_restaurants.json'], returncode=0)

Import JSON Data into MongoDB

In [12]:
mongoimport_cmd = [
    "docker", "exec", container_name, 
    "mongoimport", "--db", "TripAdvisor", "--collection", "restaurants",
    "--file", container_path, "--jsonArray"
]

subprocess.run(mongoimport_cmd, check=True)

CompletedProcess(args=['docker', 'exec', 'MONGO_CONTAINER', 'mongoimport', '--db', 'TripAdvisor', '--collection', 'restaurants', '--file', '/tmp/tripadvisor_european_restaurants.json', '--jsonArray'], returncode=0)

Show Databases and Collections

In [13]:
print("Databases:", client.list_database_names())
print("Collections in TripAdvisor:", db.list_collection_names())


Databases: ['TripAdvisor', 'admin', 'config', 'local']
Collections in TripAdvisor: ['restaurants']


Count Documents in Collection

In [14]:
print("Total Documents:", collection.count_documents({}))


Total Documents: 3250191


Show a Sample Document

In [15]:
sample_doc = collection.find_one()
pp.pprint(sample_doc)

{ '_id': ObjectId('677d4d1104e1ddf4b783318c'),
  'address': '2 rue des Dames, 36800 Rivarennes France',
  'atmosphere': None,
  'average': 0.0,
  'avg_rating': 5.0,
  'awards': [],
  'city': 'Rivarennes',
  'claimed': 'Claimed',
  'country': 'France',
  'cuisines': ['French', 'European'],
  'default_language': 'English',
  'excellent': 3.0,
  'features': [ 'Reservations',
                'Seating',
                'Table Service',
                'Wheelchair Accessible'],
  'food': None,
  'gluten_free': 'N',
  'keywords': [],
  'latitude': 46.635895,
  'longitude': 1.386133,
  'meals': ['Dinner', 'Lunch', 'Drinks'],
  'open_days_per_week': None,
  'open_hours_per_week': None,
  'original_location': [ 'Europe',
                         'France',
                         'Centre-Val de Loire',
                         'Berry',
                         'Indre',
                         'Rivarennes'],
  'original_open_hours': None,
  'poor': 0.0,
  'popularity_detailed': '#1 of 1 Restaura

Get Top 5 Vegan Breakfast Restaurants in Athens

In [16]:
Q2_1 = [
    {"$match": {"city": "Athens", "vegan_options": "Y", "meals": "Breakfast", "total_reviews_count": {"$gt": 100}}},
    {"$project": {"restaurant_name": 1, "address": 1, "avg_rating": 1, "total_reviews_count": 1, "_id": 0}},
    {"$sort": {"avg_rating": -1, "total_reviews_count": -1}},
    {"$limit": 5}
]

result = list(collection.aggregate(Q2_1))
pp.pprint(result)


[ { 'address': 'Perikleous 56 Ground floor, Athens 10560 Greece',
    'avg_rating': 5.0,
    'restaurant_name': 'Vegan Beat',
    'total_reviews_count': 754.0},
  { 'address': 'Perikleous 56 Ground floor, Athens 10560 Greece',
    'avg_rating': 5.0,
    'restaurant_name': 'Vegan Beat',
    'total_reviews_count': 754.0},
  { 'address': 'Perikleous 56 Ground floor, Athens 10560 Greece',
    'avg_rating': 5.0,
    'restaurant_name': 'Vegan Beat',
    'total_reviews_count': 754.0},
  { 'address': 'Fillelinon 22, Athens 10557 Greece',
    'avg_rating': 5.0,
    'restaurant_name': 'Victory Cafe',
    'total_reviews_count': 562.0},
  { 'address': 'Fillelinon 22, Athens 10557 Greece',
    'avg_rating': 5.0,
    'restaurant_name': 'Victory Cafe',
    'total_reviews_count': 562.0}]


Top 10 Cuisine Types for Vegan Restaurants in Athens

In [17]:
Q2_2 = [
    {"$match": {"city": "Athens", "vegan_options": "Y"}},
    {"$unwind": "$cuisines"},
    {"$group": {"_id": "$cuisines", "total_restaurants": {"$sum": 1}, "avg_reviews": {"$avg": "$total_reviews_count"}}},
    {"$sort": {"total_restaurants": -1}},
    {"$limit": 10}
]

result = list(collection.aggregate(Q2_2))
pp.pprint(result)


[ {'_id': 'Greek', 'avg_reviews': 433.1246537396122, 'total_restaurants': 1083},
  { '_id': 'Mediterranean',
    'avg_reviews': 407.1919770773639,
    'total_restaurants': 1047},
  { '_id': 'European',
    'avg_reviews': 416.4139784946237,
    'total_restaurants': 558},
  {'_id': 'Cafe', 'avg_reviews': 204.51041666666666, 'total_restaurants': 288},
  { '_id': 'Healthy',
    'avg_reviews': 674.4406779661017,
    'total_restaurants': 177},
  {'_id': 'Asian', 'avg_reviews': 278.8837209302326, 'total_restaurants': 129},
  { '_id': 'Italian',
    'avg_reviews': 197.73170731707316,
    'total_restaurants': 123},
  { '_id': 'Fast food',
    'avg_reviews': 173.8108108108108,
    'total_restaurants': 111},
  {'_id': 'Bar', 'avg_reviews': 288.05555555555554, 'total_restaurants': 108},
  { '_id': 'Seafood',
    'avg_reviews': 431.9117647058824,
    'total_restaurants': 102}]


Top 5 Cities (Outside Greece) with the Most Greek Restaurants

In [19]:
Q2_3 = [
    {"$match": {"country": {"$ne": "Greece"}, "cuisines": "Greek", "city": {"$ne": None}}},
    {"$group": {"_id": "$city", "total_restaurants": {"$sum": 1}}},
    {"$sort": {"total_restaurants": -1}},
    {"$limit": 5}
]

result = list(collection.aggregate(Q2_3))
pp.pprint(result)


[ {'_id': 'Munich', 'total_restaurants': 357},
  {'_id': 'Paris', 'total_restaurants': 315},
  {'_id': 'Nuremberg', 'total_restaurants': 213},
  {'_id': 'Dusseldorf', 'total_restaurants': 195},
  {'_id': 'Vienna', 'total_restaurants': 192}]


Best City with Most Highly Rated Restaurants

In [4]:
Q2_4 = [
    {"$match": {"avg_rating": {"$gt": 4.5}, "total_reviews_count": {"$gt": 1000}, "city": {"$ne": None}}},
    {"$group": {"_id": "$city", "total_restaurants": {"$sum": 1}}},
    {"$sort": {"total_restaurants": -1}},
    {"$limit": 1}
]

result = list(collection.aggregate(Q2_4))
pp.pprint(result)


[{'_id': 'Rome', 'total_restaurants': 10}]


Top Greek Regions with Highly Rated Restaurants

In [5]:
Q2_5 = [
    {"$match": {"country": "Greece", "avg_rating": {"$gt": 4.5}, "region": {"$ne": None}}},
    {"$group": {"_id": "$region", "total_restaurants": {"$sum": 1}}},
    {"$sort": {"total_restaurants": -1}}
]

result = list(collection.aggregate(Q2_5))
pp.pprint(result)


[ {'_id': 'Attica', 'total_restaurants': 1328},
  {'_id': 'South Aegean', 'total_restaurants': 1083},
  {'_id': 'Crete', 'total_restaurants': 1067},
  {'_id': 'Central Macedonia', 'total_restaurants': 802},
  {'_id': 'Ionian Islands', 'total_restaurants': 658},
  {'_id': 'Peloponnese', 'total_restaurants': 476},
  {'_id': 'Northeast Aegean Islands', 'total_restaurants': 337},
  {'_id': 'Thessaly', 'total_restaurants': 265},
  {'_id': 'Central Greece', 'total_restaurants': 240},
  {'_id': 'Epirus', 'total_restaurants': 231},
  {'_id': 'West Greece', 'total_restaurants': 224},
  {'_id': 'East Macedonia and Thrace', 'total_restaurants': 173},
  {'_id': 'West Macedonia', 'total_restaurants': 93},
  {'_id': 'Sporades', 'total_restaurants': 53}]


In [13]:
import time

# Function to measure query execution time
def measure_query_time(pipeline):
    start_time = time.time()
    list(collection.aggregate(pipeline))
    end_time = time.time()
    return end_time - start_time

# Define sample pipelines for testing
pipelines = [
    # Pipeline 1: Count restaurants serving vegan meals in Athens
    [
        {"$match": {"city": "Athens", "vegan_options": "Y"}},
        {"$group": {"_id": "$meals", "total_restaurants": {"$sum": 1}}},
        {"$sort": {"total_restaurants": -1}}
    ],
    # Pipeline 2: Get top 5 vegan breakfast restaurants in Athens
    [
        {"$match": {"city": "Athens", "vegan_options": "Y", "meals": "Breakfast", "total_reviews_count": {"$gt": 100}}},  
        {"$project": {"restaurant_name": 1, "address": 1, "avg_rating": 1, "total_reviews_count": 1, "_id": 0}},  
        {"$sort": {"avg_rating": -1, "total_reviews_count": -1}},  
        {"$limit": 5}  
    ],
    # Pipeline 3: Top 10 cuisine types for vegan restaurants in Athens
    [
        {"$match": {"city": "Athens", "vegan_options": "Y"}},  
        {"$unwind": "$cuisines"},  
        {"$group": {"_id": "$cuisines", "total_restaurants": {"$sum": 1}, "avg_reviews": {"$avg": "$total_reviews_count"}}},  
        {"$sort": {"total_restaurants": -1}},  
        {"$limit": 10}  
    ],
    # Pipeline 4: Top 5 cities (excluding Greece) with the most Greek restaurants
    [
        {"$match": {"country": {"$ne": "Greece"}, "cuisines": "Greek", "city": {"$ne": None}}},  
        {"$group": {"_id": "$city", "total_restaurants": {"$sum": 1}}},  
        {"$sort": {"total_restaurants": -1}},  
        {"$limit": 5}  
    ],
    # Pipeline 5: Best city with the most highly rated restaurants
    [
        {"$match": {"avg_rating": {"$gt": 4.5}, "total_reviews_count": {"$gt": 1000}, "city": {"$ne": None}}},  
        {"$group": {"_id": "$city", "total_restaurants": {"$sum": 1}}},  
        {"$sort": {"total_restaurants": -1}},  
        {"$limit": 1}  
    ]
]

# Define corresponding indexes for each pipeline
index_sets = [
    [("city", 1), ("vegan_options", 1)],  # Pipeline 1
    [("city", 1), ("vegan_options", 1), ("meals", 1), ("total_reviews_count", -1)],  # Pipeline 2
    [("city", 1), ("vegan_options", 1), ("cuisines", 1)],  # Pipeline 3
    [("country", 1), ("cuisines", 1), ("city", 1)],  # Pipeline 4
    [("avg_rating", -1), ("total_reviews_count", -1), ("city", 1)]  # Pipeline 5
]

# Execute and measure performance for each pipeline
performance_results = []

for i, (pipeline, indexes) in enumerate(zip(pipelines, index_sets)):
    print(f"\nRunning Pipeline {i+1}")

    # Drop previous indexes
    collection.drop_indexes()
    print("Previous indexes deleted")

    # Measure query time BEFORE indexing
    time_before = measure_query_time(pipeline)
    
    # Create new indexes
    for index in indexes:
        collection.create_index([index])
    
    print("New indexes created")

    # Measure query time AFTER indexing
    time_after = measure_query_time(pipeline)

    # Store results for later summary
    performance_results.append((i+1, time_before, time_after))

# Print final performance summary
print("\nQuery Performance Summary:")
print(f"{'Pipeline':<10}{'Before Indexing (s)':<25}{'After Indexing (s)':<25}{'Speedup':<10}")
print("-" * 70)
for pipeline_id, before, after in performance_results:
    speedup = before / after if after > 0 else float("inf")
    print(f"{pipeline_id:<10}{before:<25.4f}{after:<25.4f}{speedup:.2f}x Faster")




Running Pipeline 1
Previous indexes deleted
New indexes created

Running Pipeline 2
Previous indexes deleted
New indexes created

Running Pipeline 3
Previous indexes deleted
New indexes created

Running Pipeline 4
Previous indexes deleted
New indexes created

Running Pipeline 5
Previous indexes deleted
New indexes created

Query Performance Summary:
Pipeline  Before Indexing (s)      After Indexing (s)       Speedup   
----------------------------------------------------------------------
1         0.7831                   0.0114                   68.88x Faster
2         0.4628                   0.0367                   12.62x Faster
3         0.4734                   0.0123                   38.44x Faster
4         0.8302                   0.0654                   12.69x Faster
5         0.8095                   0.0746                   10.86x Faster
